import & setup cơ bản

In [3]:
import pandas as pd
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)

ĐỌC BÁO CÁO TỪ FILE CLEAN

In [4]:
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
file_BCTC = path_bctc_clean_quarterly + 'BCTC_quarterly.xlsx'
file_BCTC_none_bds_airline_bank = path_bctc_clean_quarterly + "BCTC_none_bds_airline_bank_quarterly.xlsx"
BCTC = pd.read_excel(file_BCTC,sheet_name=0, header=0)
#BCTC_none_bds_airline_bank = pd.read_excel(file_BCTC_none_bds_airline_bank,sheet_name=0, header=0)

Trả lời câu hỏi diễn biến lợi nhuận của toàn thị trường

In [5]:
BCTC['EBITDA'] = BCTC['15. Profit before tax (50=30+40+41)'] + BCTC['Depreciation of fixed assets and properties investment'] -  BCTC['7. Financial expenses'] # thêm EBITDA
sector_exclude = ['Real Estate Development','Scheduled Air Transportation','Depository Credit Intermediation']

BCTC_none_bds_airline_bank = BCTC[~BCTC['Sector (NAICS) Level 3'].isin(sector_exclude)] 

In [6]:
BCTC.groupby(['Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA']].sum()

,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52),EBITDA
Year_Q,,,,,,
2022Q1,"17,031,477,863,196,608.00","3,291,871,813,564,999.00","1,825,124,966,378,724.00",167262839443628,137226872075364,"99,473,536,831,288.00"
2022Q2,"17,731,469,166,798,362.00","3,383,672,388,204,010.00","1,877,735,420,774,621.00",161068144538761,131066013242561,"97,188,894,068,585.00"
2022Q3,"18,113,531,705,129,560.00","3,479,473,906,160,057.00","1,925,001,723,216,154.00",147127017024150,113163186994761,"83,434,325,657,444.00"
2022Q4,"18,738,564,748,796,332.00","3,536,152,317,779,585.00","1,986,116,215,655,189.00",111613984695822,82437492201068,"59,173,646,124,403.00"
2023Q1,"19,053,721,647,011,264.00","3,622,428,715,297,055.00","1,984,567,307,307,387.00",140243944355003,108922405390016,"73,629,490,333,577.00"
2023Q2,"19,384,641,583,005,364.00","3,677,106,829,211,450.00","2,009,076,065,073,154.00",144437939945698,112882152640934,"81,569,098,424,649.00"
2023Q3,"19,790,989,523,580,272.00","3,772,124,637,010,962.00","2,070,503,788,663,954.00",138593566886196,107041136654780,"65,859,380,759,435.00"
2023Q4,"21,030,537,527,000,124.00","3,880,740,434,052,606.00","2,129,959,464,807,399.00",141766043990823,112748459352408,"75,290,296,602,996.00"
2024Q1,"21,174,986,011,275,804.00","3,994,688,084,942,986.00","2,129,061,292,313,091.00",145179102049220,116078602526532,"67,896,584,234,435.00"


In [7]:
BCTC['Market_cap'] = BCTC["1. Owner's capital "]*BCTC['Close Price End of period']/10000
Exchange_BCTC = BCTC.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']].sum()
Exchange_BCTC['EPS'] = Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC["1. Owner's capital "]/10000)
Exchange_BCTC['PE_quarter'] = Exchange_BCTC['Market_cap']/(Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC['Rolling_Profit_4Q'] = Exchange_BCTC.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC['Rolling_Profit_4Q']/Exchange_BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC['ROE(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["1. Owner's capital "]
Exchange_BCTC['ROA(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC["Leverage"] = Exchange_BCTC["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC["B. OWNER'S EQUITY (400=410+430)"]
Exchange_BCTC['Rolling_EPS_4Q'] = Exchange_BCTC.groupby('Exchange')['EPS'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Rolling_PE_4Q'] = Exchange_BCTC['Market_cap']/Exchange_BCTC['Rolling_Profit_4Q']
Exchange_BCTC

TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      663,873,396,686,762.00           213,560,249,460,540.00   
         2022Q2      673,732,816,880,137.00           215,997,026,588,739.00   
         2022Q3      679,192,290,945,222.00           220,427,913,266,414.00   
         2022Q4      701,310,886,809,281.00           225,971,574,483,550.00   
         2023Q1      701,313,648,175,627.00           228,300,014,208,221.00   
         2023Q2      704,917,168,995,262.00           227,907,497,989,355.00   
         2023Q3      745,819,821,049,330.00           242,078,494,289,033.00   
         2023Q4      768,885,982,070,875.00           246,399,072,376,082.00   
         2024Q1      770,586,723,025,712.00           251,386,959,411,251.00   
HOSE     2022Q1   14,651,826,458,259,566.00         2,508,463,162,960,834.00   
         2022Q2   15,301,400,237,432,612.00         2,583,556,665,374,741.00   
         2022Q3   15,685,771,912,209,656.00         2,667,793,512,143,241.00   
         2022Q4   16,307,603,148,228,428.00         2,718,715,710,984,809.00   
         2023Q1   16,663,463,786,584,980.00         2,797,510,024,373,828.00   
         2023Q2   16,928,654,410,260,620.00         2,853,837,437,247,078.00   
         2023Q3   17,253,804,929,636,742.00         2,915,987,481,316,900.00   
         2023Q4   18,392,993,003,093,656.00         3,011,169,390,540,611.00   
         2024Q1   18,561,584,916,060,008.00         3,111,192,062,182,978.00   
UPCOM    2022Q1    1,715,778,008,250,279.00           569,848,401,143,625.00   
         2022Q2    1,756,336,112,485,612.00           584,118,696,240,530.00   
         2022Q3    1,748,567,501,974,682.00           591,252,480,750,402.00   
         2022Q4    1,729,650,713,758,623.00           591,465,032,311,226.00   
         2023Q1    1,688,944,212,250,660.00           596,618,676,715,006.00   
         2023Q2    1,751,070,003,749,481.00           595,361,893,975,017.00   
         2023Q3    1,791,364,772,894,199.00           614,058,661,405,029.00   
         2023Q4    1,868,658,541,835,595.00           623,171,971,135,913.00   
         2024Q1    1,842,814,372,190,083.00           632,109,063,348,757.00   

                     1. Owner's capital   15. Profit before tax (50=30+40+41)  \
Exchange Year_Q                                                                 
HNX      2022Q1   128,555,038,621,729.00                        7216462299893   
         2022Q2   133,421,873,420,000.00                        6455456774879   
         2022Q3   140,022,487,073,000.00                        4613336530290   
         2022Q4   143,267,213,799,157.00                        5828427622700   
         2023Q1   143,217,787,910,000.00                        4874058245684   
         2023Q2   143,820,079,380,000.00                        6218255708156   
         2023Q3   156,048,368,050,800.00                        4626003397309   
         2023Q4   157,762,416,520,800.00                        6641833958970   
         2024Q1   158,308,247,459,502.00                        7008886959760   
HOSE     2022Q1 1,267,898,611,797,831.00                      138514978954291   
         2022Q2 1,316,166,288,767,831.00                      120945237565462   
         2022Q3 1,355,044,061,087,831.00                      125237179448302   
         2022Q4 1,415,118,821,024,242.00                       93184148115606   
         2023Q1 1,419,779,487,537,832.00                      118101679784116   
         2023Q2 1,439,327,682,987,831.00                      121143043023949   
         2023Q3 1,480,434,640,577,831.00                      112282181768887   
         2023Q4 1,530,523,486,917,810.00                      113895408730510   
         2024Q1 1,529,944,975,677,831.00                      119212152881205   
UPCOM    2022Q1   428,671,315,959,164.00                       2153139818944

In [8]:
BCTC_none_bds_airline_bank.groupby(['Year_Q'])[['3. Net revenue  (10 = 01 - 03)','7. Financial expenses','15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)']].sum()

,3. Net revenue (10 = 01 - 03),7. Financial expenses,15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52)
Year_Q,,,,
2022Q1,"847,291,776,009,728.00","19,425,089,156,368.00",89177358368257,76129146345684
2022Q2,"939,673,338,985,084.00","29,026,362,376,851.00",91260665285704,77696068055007
2022Q3,"869,264,375,659,732.00","25,713,926,330,826.00",60229175064032,49168843970390
2022Q4,"900,545,391,900,858.00","30,219,441,806,485.00",42969353178014,33253131889999
2023Q1,"760,895,622,089,080.00","26,261,006,898,637.00",51921923728190,41993730747186
2023Q2,"824,198,583,381,975.00","27,441,765,103,855.00",63469768902973,51424871934421
2023Q3,"824,942,213,400,334.00","28,550,839,605,964.00",58743101147702,47799226608045
2023Q4,"904,422,583,020,785.00","24,831,147,795,037.00",65684572896851,53702294071763
2024Q1,"811,814,775,333,555.00","22,643,077,353,688.00",61699831805790,50179206347350


TRẢ LỜI CHO CÂU HỎI THỊ TRƯỜNG NONE BANK, REAL ESTATE, AIRLINE

In [9]:
BCTC_none_bds_airline_bank['Market_cap'] = BCTC_none_bds_airline_bank["1. Owner's capital "]*BCTC_none_bds_airline_bank['Close Price End of period']/10000
Exchange_BCTC_None = BCTC_none_bds_airline_bank.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum()
Exchange_BCTC_None['EPS'] = Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC_None["1. Owner's capital "]/10000)
Exchange_BCTC_None['PE_quarter'] = Exchange_BCTC_None['Market_cap']/(Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC_None['Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC_None['Rolling_PE_4Q'] = Exchange_BCTC_None['Market_cap']/Exchange_BCTC_None['Rolling_Profit_4Q']
Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC_None['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC_None['Rolling_Profit_4Q']/Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC_None['ROE(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["1. Owner's capital "]
Exchange_BCTC_None['ROA(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC_None["Leverage"] = Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC_None["B. OWNER'S EQUITY (400=410+430)"]

Exchange_BCTC_None

C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_804\3043153505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BCTC_none_bds_airline_bank['Market_cap'] = BCTC_none_bds_airline_bank["1. Owner's capital "]*BCTC_none_bds_airline_bank['Close Price End of period']/10000


TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      433,165,036,763,814.00           180,858,622,663,242.00   
         2022Q2      432,589,695,933,622.00           183,248,985,408,500.00   
         2022Q3      435,392,197,213,879.00           185,791,367,385,272.00   
         2022Q4      441,261,330,172,538.00           190,081,767,756,358.00   
         2023Q1      443,770,827,612,879.00           191,647,177,700,040.00   
         2023Q2      441,254,970,318,703.00           190,628,689,631,697.00   
         2023Q3      461,364,582,917,943.00           202,251,215,600,838.00   
         2023Q4      470,943,463,956,578.00           205,767,713,588,154.00   
         2024Q1      476,029,324,663,180.00           210,022,069,131,522.00   
HOSE     2022Q1    2,681,856,972,974,357.00         1,168,156,575,629,444.00   
         2022Q2    2,743,509,770,585,992.00         1,206,086,759,860,550.00   
         2022Q3    2,725,949,063,993,746.00         1,224,408,766,838,058.00   
         2022Q4    2,716,407,754,602,996.00         1,233,957,391,807,090.00   
         2023Q1    2,733,810,226,766,623.00         1,254,555,587,708,308.00   
         2023Q2    2,788,917,192,203,483.00         1,268,054,656,181,381.00   
         2023Q3    2,781,980,711,113,190.00         1,262,857,038,286,082.00   
         2023Q4    2,836,133,811,235,337.00         1,266,360,869,156,994.00   
         2024Q1    2,902,761,918,879,155.00         1,303,408,734,609,078.00   
UPCOM    2022Q1    1,080,850,277,515,437.00           498,456,268,379,800.00   
         2022Q2    1,102,251,326,905,101.00           511,836,706,902,646.00   
         2022Q3    1,085,531,319,355,943.00           518,089,077,134,210.00   
         2022Q4    1,076,965,331,623,282.00           524,858,650,186,213.00   
         2023Q1    1,058,825,377,583,138.00           529,125,927,476,567.00   
         2023Q2    1,073,191,383,691,283.00           525,955,165,298,981.00   
         2023Q3    1,101,173,043,977,792.00           538,490,935,848,324.00   
         2023Q4    1,112,220,495,643,916.00           542,124,484,475,372.00   
         2024Q1    1,109,568,387,552,730.00           549,447,194,263,781.00   

                   1. Owner's capital   III. Short-term receivables  \
Exchange Year_Q                                                       
HNX      2022Q1 105,335,176,751,729.00       105,319,413,352,992.00   
         2022Q2 110,189,413,710,000.00       109,986,946,886,715.00   
         2022Q3 113,873,299,683,000.00       111,293,639,124,411.00   
         2022Q4 116,667,176,369,157.00       113,439,971,020,960.00   
         2023Q1 116,612,815,480,000.00       113,039,095,731,353.00   
         2023Q2 116,958,494,160,000.00       114,627,773,083,224.00   
         2023Q3 126,498,148,980,800.00       118,333,741,120,124.00   
         2023Q4 127,667,388,400,800.00       119,669,999,335,543.00   
         2024Q1 127,588,149,339,502.00       118,486,290,100,927.00   
HOSE     2022Q1 604,995,284,427,831.00       401,419,408,370,821.00   
         2022Q2 641,560,868,877,831.00       422,881,723,227,763.00   
         2022Q3 668,523,454,007,831.00       418,357,077,167,679.00   
         2022Q4 679,201,181,344,242.00       412,750,431,252,935.00   
         2023Q1 680,913,275,937,831.00       440,727,047,921,832.00   
         2023Q2 684,545,758,117,831.00       445,783,012,726,731.00   
         2023Q3 690,846,242,277,831.00       428,473,434,397,447.00   
         2023Q4 696,751,293,417,810.00       398,728,659,886,614.00   
         2024Q1 700,121,501,957,831.00       431,339,418,384,740.00   
UPCOM    2022Q1 374,785,650,749,164.00       217,769,796,249,359.00   
         2022Q2 375,161,593,376,790.00       240,359,393,292,944.00   
         2022Q3 375,967,609,925,323.00       229,366,155,963,337.00   
         2022Q4 380,849,093,771,790.00       221,289,889,038,39

ĐÁNH GIÁ PE THEO NGÀNH

In [10]:
Sector_BCTC = BCTC.groupby(['Sector (NAICS) Level 3','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum()
Sector_BCTC['EPS_quarter'] = Sector_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Sector_BCTC["1. Owner's capital "]/10000)
Sector_BCTC['Rolling_EPS_4Q'] = Sector_BCTC.groupby('Sector (NAICS) Level 3')['EPS_quarter'].rolling(4).sum().reset_index(level=0, drop=True)
Sector_BCTC['PE_quarter'] = Sector_BCTC['Market_cap']/(Sector_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Sector_BCTC['Rolling_Profit_4Q'] = Sector_BCTC.groupby('Sector (NAICS) Level 3')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Sector_BCTC['Last_Year_Rolling_Profit_4Q'] = Sector_BCTC.groupby(['Sector (NAICS) Level 3'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Sector_BCTC['YoY_Growth_Rolling_Profit_4Q'] = (Sector_BCTC['Rolling_Profit_4Q']/Sector_BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
Sector_BCTC['ROE(%)'] = Sector_BCTC['Rolling_Profit_4Q']*100/Sector_BCTC["1. Owner's capital "]
Sector_BCTC['ROA(%)'] = Sector_BCTC['Rolling_Profit_4Q']*100/Sector_BCTC["TOTAL ASSETS (280=100+200)"]
Sector_BCTC['Rolling_PE_4Q'] = Sector_BCTC['Market_cap']/Sector_BCTC['Rolling_Profit_4Q']
Sector_BCTC

TOTAL ASSETS (280=100+200)  \
Sector (NAICS) Level 3                             Year_Q                               
Activities Related to Real Estate                  2022Q1       36,886,685,505,607.00   
                                                   2022Q2       39,813,229,147,977.00   
                                                   2022Q3       40,228,449,030,189.00   
                                                   2022Q4       39,916,305,596,922.00   
                                                   2023Q1       37,038,487,501,918.00   
...                                                                               ...   
Wireless Telecommunications Carriers (except Sa... 2023Q1       50,502,804,475,633.00   
                                                   2023Q2       48,424,867,370,960.00   
                                                   2023Q3       50,803,353,975,670.00   
                                                   2023Q4       52,371,351,113,757.00   
                                                   2024Q1       55,367,853,104,821.00   

                                                           B. OWNER'S EQUITY (400=410+430)  \
Sector (NAICS) Level 3                             Year_Q                                    
Activities Related to Real Estate                  2022Q1            17,018,678,240,094.00   
                                                   2022Q2            18,156,284,425,853.00   
                                                   2022Q3            18,262,564,043,043.00   
                                                   2022Q4            18,092,669,921,343.00   
                                                   2023Q1            17,775,555,033,441.00   
...                                                                                    ...   
Wireless Telecommunications Carriers (except Sa... 2023Q1            29,690,953,206,043.00   
                                                   2023Q2            28,750,124,791,181.00   
                                                   2023Q3            29,914,708,205,697.00   
                                                   2023Q4            30,362,756,269,123.00   
                                                   2024Q1            32,114,298,771,192.00   

                                                            1. Owner's capital   \
Sector (NAICS) Level 3                             Year_Q                         
Activities Related to Real Estate                  2022Q1 14,005,665,750,000.00   
                                                   2022Q2 15,025,578,580,000.00   
                                                   2022Q3 15,083,437,120,000.00   
                                                   2022Q4 15,411,034,970,000.00   
                                                   2023Q1 15,187,912,550,000.00   
...                                                                         ...   
Wireless Telecommunications Carriers (except Sa... 2023Q1 30,438,112,000,000.00   
                                                   2023Q2 30,438,112,000,000.00   
                                                   2023Q3 30,438,112,000,000.00   
                                                   2023Q4 30,438,112,000,000.00   
                                                   2024Q1 30,438,112,000,000.00   

                                                           III. Short-term receivables  \
Sector (NAICS) Level 3                             Year_Q                                
Activities Related to Real Estate                  2022Q1        12,515,060,279,546.00   
                                                   2022Q2        13,080,506,965,686.00   
                                                   2022Q3        12,982,177,628,494.00   
                                                   2022Q4        11,704,906,262,848.00   
                                                   2023Q1 

Chọn Sector

In [14]:
Choose_sector = "Cut and Sew Apparel Manufacturing"
Choose_sector_overview = Sector_BCTC.loc[[Choose_sector]]
Choose_sector_overview

TOTAL ASSETS (280=100+200)  \
Sector (NAICS) Level 3            Year_Q                               
Cut and Sew Apparel Manufacturing 2022Q1       23,555,649,616,883.00   
                                  2022Q2       25,066,476,232,786.00   
                                  2022Q3       25,428,144,020,547.00   
                                  2022Q4       25,002,380,662,482.00   
                                  2023Q1       24,036,777,818,335.00   
                                  2023Q2       24,264,810,185,869.00   
                                  2023Q3       23,892,010,200,004.00   
                                  2023Q4       23,920,226,682,989.00   
                                  2024Q1       23,594,656,528,717.00   

                                          B. OWNER'S EQUITY (400=410+430)  \
Sector (NAICS) Level 3            Year_Q                                    
Cut and Sew Apparel Manufacturing 2022Q1             9,157,361,424,238.00   
                                  2022Q2             9,294,090,202,300.00   
                                  2022Q3            10,223,478,833,990.00   
                                  2022Q4            10,312,705,378,552.00   
                                  2023Q1            10,216,059,305,466.00   
                                  2023Q2            10,000,475,652,624.00   
                                  2023Q3            10,233,302,555,239.00   
                                  2023Q4            10,328,922,687,013.00   
                                  2024Q1            10,492,393,362,877.00   

                                          1. Owner's capital   \
Sector (NAICS) Level 3            Year_Q                        
Cut and Sew Apparel Manufacturing 2022Q1 4,620,682,920,000.00   
                                  2022Q2 5,051,726,830,000.00   
                                  2022Q3 5,527,639,920,000.00   
                                  2022Q4 5,697,255,760,001.00   
                                  2023Q1 5,534,785,760,000.00   
                                  2023Q2 5,534,785,760,000.00   
                                  2023Q3 5,731,864,340,000.00   
                                  2023Q4 5,810,630,210,000.00   
                                  2024Q1 5,810,297,729,502.00   

                                          III. Short-term receivables  \
Sector (NAICS) Level 3            Year_Q                                
Cut and Sew Apparel Manufacturing 2022Q1         4,150,967,483,423.00   
                                  2022Q2         4,852,747,487,064.00   
                                  2022Q3         4,247,045,007,431.00   
                                  2022Q4         3,606,132,136,275.00   
                                  2023Q1         3,559,223,105,088.00   
                                  2023Q2         3,973,824,716,485.00   
                                  2023Q3         4,270,756,571,318.00   
                                  2023Q4         4,312,636,621,354.00   
                                  2024Q1         3,828,077,536,173.00   

                                              IV. Inventories  \
Sector (NAICS) Level 3            Year_Q                        
Cut and Sew Apparel Manufacturing 2022Q1 6,425,534,534,145.00   
                                  2022Q2 7,242,736,764,287.00   
                                  2022Q3 6,064,562,939,668.00   
                                  2022Q4 6,112,011,221,505.00   
                                  2023Q1 6,450,281,465,685.00   
                                  2023Q2 6,252,492,437,181.00   
                                  2023Q3 4,981,612,436,229.00   
                                  2023Q4 5,223,436,506,642.00   
                                  2024Q1 5,685,171,363,761.00   

                                          15. Profit before tax (50=30+40+41)  \
Sector (NAICS) Level 3            Year_Q                                        
Cut

In [12]:
Choose_sector_detailview = BCTC[BCTC['Sector (NAICS) Level 3']==Choose_sector][["Stock code","Year_Q","TOTAL ASSETS (280=100+200)","1. Owner's capital ",'18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']]
Choose_sector_detailview['Last_Year_Quarter_Profit'] = Choose_sector_detailview.groupby(["Stock code"])['18. Net profit after tax  (60 = 50 - 51 - 52)'].shift(4)  #lay loi nhuan quy cung ky nam truoc
Choose_sector_detailview['YoY_Growth_Quarter'] = (Choose_sector_detailview['18. Net profit after tax  (60 = 50 - 51 - 52)']/Choose_sector_detailview['Last_Year_Quarter_Profit']-1)*100
Choose_sector_detailview['EPS_quarter'] = Choose_sector_detailview['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Choose_sector_detailview["1. Owner's capital "]/10000)
Choose_sector_detailview['Rolling_EPS_4Q'] = Choose_sector_detailview.groupby('Stock code')['EPS_quarter'].rolling(4).sum().reset_index(level=0, drop=True)
Choose_sector_detailview['Rolling_Profit_4Q'] = Choose_sector_detailview.groupby('Stock code')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Choose_sector_detailview['Last_Year_Rolling_Profit_4Q'] = Choose_sector_detailview.groupby(["Stock code"])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Choose_sector_detailview['YoY_Growth_Rolling_Profit_4Q'] = (Choose_sector_detailview['Rolling_Profit_4Q']/Choose_sector_detailview['Last_Year_Rolling_Profit_4Q']-1)*100
Choose_sector_detailview['Rolling_PE_4Q'] = Choose_sector_detailview['Market_cap']/Choose_sector_detailview['Rolling_Profit_4Q']
Choose_sector_detailview['ROE(%)'] = Choose_sector_detailview['Rolling_Profit_4Q']*100/Choose_sector_detailview["1. Owner's capital "]
Choose_sector_detailview['ROA(%)'] = Choose_sector_detailview['Rolling_Profit_4Q']*100/Choose_sector_detailview["TOTAL ASSETS (280=100+200)"]
Choose_sector_detailview

,Stock code,Year_Q,TOTAL ASSETS (280=100+200),1. Owner's capital,18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Last_Year_Quarter_Profit,YoY_Growth_Quarter,EPS_quarter,Rolling_EPS_4Q,Rolling_Profit_4Q,Last_Year_Rolling_Profit_4Q,YoY_Growth_Rolling_Profit_4Q,Rolling_PE_4Q,ROE(%),ROA(%)
3,AAT,2022Q1,"1,237,793,386,803.00","638,014,890,000.00",21407935,"874,080,399,300.00",NaN,NaN,0.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,BDG,2022Q1,"1,008,703,244,019.00","120,000,000,000.00",63736837864,"543,600,000,000.00",NaN,NaN,"5,311.40",NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,DM7,2022Q1,"377,282,687,306.00","154,111,000,000.00",16802262971,"130,994,350,000.00",NaN,NaN,"1,090.27",NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,GMC,2022Q1,"796,161,936,295.00","330,002,590,000.00",-8106918630,"919,057,213,150.00",NaN,NaN,-245.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,HSM,2022Q1,"2,051,121,204,772.00","205,000,000,000.00",36645051586,"446,900,000,000.00",NaN,NaN,"1,787.56",NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,HUG,2022Q1,"672,404,300,943.00","162,597,250,000.00",23592279041,"650,389,000,000.00",NaN,NaN,"1,450.96",NaN,NaN,NaN,NaN,NaN,NaN,NaN
541,M10,2022Q1,"1,994,152,245,091.00","302,400,000,000.00",22855583294,"728,784,000,000.00",NaN,NaN,755.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,MNB,2022Q1,"2,777,679,511,277.00","182,000,000,000.00",20304037045,"418,600,000,000.00",NaN,NaN,"1,115.61",NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,MSH,2022Q1,"3,267,346,195,701.00","500,094,000,000.00",81889457342,"4,350,817,800,000.00",NaN,NaN,"1,637.48",NaN,NaN,NaN,NaN,NaN,NaN,NaN
911,TCM,2022Q1,"3,750,006,009,701.00","713,608,080,000.00",73679979563,"5,387,741,004,000.00",NaN,NaN,"1,032.50",NaN,NaN,NaN,NaN,NaN,NaN,NaN
